In [ ]:
from __future__ import print_function
import torch
from torch.optim.optimizer import Optimizer, required
import torch.nn.functional as F
#%matplotlib inline
import argparse
import os
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Root directory for dataset
dataroot = "data/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 32

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 15

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# create ./data under current directory
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
# For calculating fid 
testloader = torch.utils.data.DataLoader(testset,batch_size=1000,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(trainloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
# Spectral Normalization Implementation
# Alex.G
# As mentioned in page 15 of the below paper
# https://arxiv.org/pdf/1802.05957
# 

def spectralNorm(weight):
    # For conv2d layers, weight matrix is of dim [C_in, C_out, H, W]
    # We need to recast the matrix as [C_out, C_in*H*W]
    C_in = weight.size()[0]
    C_out = weight.size()[1]
    H = weight.size()[2]
    W = weight.size()[3]
    
    weight = weight.permute(1,0,2,3).reshape(C_out,C_in*H*W) # dim [C_out, C_in*H*W]
    
    u = torch.randn(C_out,1) # random normal vector
    weight_t = torch.transpose(weight,0,1)
    weight_t_u = torch.matmul(weight_t,u)
    v = weight_t_u/torch.norm(weight_t_u) 
    weight_t_v = torch.matmul(weight,v)
    u = weight_t_v/torch.norm(weight_t_v)
    
    sigma = torch.matmul(torch.matmul(torch.transpose(u,0,1),weight),v)
    # for normalized weights sigma = 1
    
    sn_weight = weight/sigma
    return nn.Parameter(sn_weight.view(C_out, C_in, H, W).permute(1,0,2,3))

In [ ]:
#
# SA Module Implementation
# Alex.G
# Implementation of Self-Attention layer as presented in the 
# SAGAN paper arXiv:1805.08318

class SelfAttention(torch.nn.Module):
    # C - No of input channels
    # k - Attention size
    def __init__(self, C, k):
        super(SelfAttention,self).__init__()
        self.C = C
        self.C_bar = C//k
        self.gamma = torch.nn.Parameter(torch.tensor(0.))
        
        self.g = torch.nn.Conv2d(C,self.C_bar,1,stride=1)
        self.f = torch.nn.Conv2d(C,self.C_bar,1,stride=1)
        self.h = torch.nn.Conv2d(C,self.C_bar,1,stride=1)
        self.v = torch.nn.Conv1d(self.C_bar,C,1,stride=1)
        
    def forward(self, x):
        
        # x is of dim [N,C,H,W]
        N = x.size()[0]
        H = x.size()[2]
        W = x.size()[3]
        
        self.f.weight = spectralNorm(self.f.weight) 
        f = self.f(x)
        # f = [N,C/k,H,W] dimension, we need to flatten to [N,C/k,H*W]
        f = f.view(N,self.C_bar,H*W)
        
        self.g.weight = spectralNorm(self.g.weight) 
        g = self.g(x)
        g = g.view(N,self.C_bar,H*W)
        
        
        # s = tranpose(f(x))*g(x)
        # s is dim [N,H*W,H*W]
        s = torch.matmul(torch.transpose(f,1,2),g)
        
        # Beta matrix transpose
        # Beta dim [N,H*W,H*W]
        B_t = F.softmax(s,dim=1)
        
        # flatten h
        self.h.weight = spectralNorm(self.h.weight) 
        h = self.h(x)
        h = h.view(N,self.C_bar,H*W) # dim [N,C/k,H*W]
        
        # o is dim [B,C,H*W] 
        o = self.v(torch.matmul(h,B_t))
        
        return (self.gamma*o + x.view(N,self.C,H*W)).view(N,self.C,H,W)


In [ ]:
# fid 1 implementation
# Implementation of fid code by the TA - Marissa Dominijanni

from scipy import linalg
from tqdm import tqdm_notebook as tqdm
import torchvision.models as models
import warnings
import math

class FID():
    def __init__(self, cache_dir, device='cpu', transform_input=True):
        #cuda:0
        os.environ["TORCH_HOME"] = "./Cache"
        self.device=device
        self.transform_input = transform_input
        self.InceptionV3 = models.inception_v3(pretrained=True, transform_input=False, aux_logits=False).to(device=self.device)
        self.InceptionV3.eval()
    
    def build_maps(self, x):
        # Resize to Fit InceptionV3
        if list(x.shape[-2:]) != [299,299]:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                x = F.interpolate(x, size=[299,299], mode='bilinear')
        # Transform Input to InceptionV3 Standards
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
            #print(x.size())
        # Run Through Partial InceptionV3 Model
        with torch.no_grad():
            # N x 3 x 299 x 299
            x = self.InceptionV3.Conv2d_1a_3x3(x)
            # N x 32 x 149 x 149
            x = self.InceptionV3.Conv2d_2a_3x3(x)
            # N x 32 x 147 x 147
            x = self.InceptionV3.Conv2d_2b_3x3(x)
            # N x 64 x 147 x 147
            x = F.max_pool2d(x, kernel_size=3, stride=2)
            # N x 64 x 73 x 73
            x = self.InceptionV3.Conv2d_3b_1x1(x)
            # N x 80 x 73 x 73
            x = self.InceptionV3.Conv2d_4a_3x3(x)
            # N x 192 x 71 x 71
            x = F.max_pool2d(x, kernel_size=3, stride=2)
            # N x 192 x 35 x 35
            x = self.InceptionV3.Mixed_5b(x)
            # N x 256 x 35 x 35
            x = self.InceptionV3.Mixed_5c(x)
            # N x 288 x 35 x 35
            x = self.InceptionV3.Mixed_5d(x)
            # N x 288 x 35 x 35
            x = self.InceptionV3.Mixed_6a(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6b(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6c(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6d(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6e(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_7a(x)
            # N x 1280 x 8 x 8
            x = self.InceptionV3.Mixed_7b(x)
            # N x 2048 x 8 x 8
            x = self.InceptionV3.Mixed_7c(x)
            # N x 2048 x 8 x 8
            # Adaptive average pooling
            x = F.adaptive_avg_pool2d(x, (1, 1))
            # N x 2048 x 1 x 1
            return x
    
    def compute_fid(self, real_images, generated_images, batch_size=128):
        # Ensure Set Sizes are the Same
        assert(real_images.shape[0] == generated_images.shape[0])
        # Build Random Sampling Orders
        real_images = real_images[np.random.permutation(real_images.shape[0])]
        generated_images = generated_images[np.random.permutation(generated_images.shape[0])]
        # Lists of Maps per Batch
        real_maps = []
        generated_maps = []
        # Build Maps
        for s in tqdm(range(math.ceil(real_images.shape[0]/batch_size)), desc='Evaluation', leave=False):
            sidx = np.arange(batch_size*s, min(batch_size*(s+1), real_images.shape[0]))
            real_maps.append(self.build_maps(real_images[sidx].to(device=self.device)).detach().to(device='cpu'))
            generated_maps.append(self.build_maps(generated_images[sidx].to(device=self.device)).detach().to(device='cpu'))
        
        # Concatenate Maps
        real_maps = np.squeeze(torch.cat(real_maps).numpy())
        generated_maps = np.squeeze(torch.cat(generated_maps).numpy())
        #print(real_maps.shape)
        #print(generated_maps.shape)
        # Calculate FID
        # Activation Statistics
        mu_g = np.mean(generated_maps, axis=0)
        mu_x = np.mean(real_maps, axis=0)
        #print('means')
        #print(mu_g.shape,mu_x.shape)
        sigma_g = np.cov(generated_maps, rowvar=False)
        sigma_x = np.cov(real_maps, rowvar=False)
        #print('covar')
        #print(sigma_g.shape,sigma_x.shape)
        # Sum of Squared Differences
        ssd = np.sum((mu_g - mu_x)**2)
        # Square Root of Product of Covariances
        covmean = linalg.sqrtm(sigma_g.dot(sigma_x), disp=False)[0]
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        # Final FID Computation
        #print(ssd)
        t = np.trace(sigma_g + sigma_x - 2*covmean)
        #print(t)
        return ssd + t

In [ ]:
# Alex. G
# As per paper by H.Zhang https://arxiv.org/abs/1706.08500
# fid 2nd implementation

class fID():
    def __init__(self,device='cpu'):
        self.device = device
        self.InceptionV3 = torchvision.models.inception_v3(pretrained=True,
                                                           transform_input=False, aux_logits=False).to(device=self.device)
        self.InceptionV3.eval()
        
    def resize_to_incpV3(self,x):
        # resize to N*2048*1*1 as per incV3
        if x.size()[-2:] != [299.299]:
            x = F.interpolate(x,size=[299,299],mode='bilinear')
            x_c1 = torch.unsqueeze(x[:,0],1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5 
            x_c2 = torch.unsqueeze(x[:,1],1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_c3 = torch.unsqueeze(x[:,2],1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_c1,x_c2,x_c3),1)
            
            # forward pass through
            with torch.no_grad():
                # N x 3 x 299 x 299
                x = self.InceptionV3.Conv2d_1a_3x3(x)
                # N x 32 x 149 x 149
                x = self.InceptionV3.Conv2d_2a_3x3(x)
                # N x 32 x 147 x 147
                x = self.InceptionV3.Conv2d_2b_3x3(x)
                # N x 64 x 147 x 147
                x = F.max_pool2d(x, kernel_size=3, stride=2)
                # N x 64 x 73 x 73
                x = self.InceptionV3.Conv2d_3b_1x1(x)
                # N x 80 x 73 x 73
                x = self.InceptionV3.Conv2d_4a_3x3(x)
                # N x 192 x 71 x 71
                x = F.max_pool2d(x, kernel_size=3, stride=2)
                # N x 192 x 35 x 35
                x = self.InceptionV3.Mixed_5b(x)
                # N x 256 x 35 x 35
                x = self.InceptionV3.Mixed_5c(x)
                # N x 288 x 35 x 35
                x = self.InceptionV3.Mixed_5d(x)
                # N x 288 x 35 x 35
                x = self.InceptionV3.Mixed_6a(x)
                # N x 768 x 17 x 17
                x = self.InceptionV3.Mixed_6b(x)
                # N x 768 x 17 x 17
                x = self.InceptionV3.Mixed_6c(x)
                # N x 768 x 17 x 17
                x = self.InceptionV3.Mixed_6d(x)
                # N x 768 x 17 x 17
                x = self.InceptionV3.Mixed_6e(x)
                # N x 768 x 17 x 17
                x = self.InceptionV3.Mixed_7a(x)
                # N x 1280 x 8 x 8
                x = self.InceptionV3.Mixed_7b(x)
                # N x 2048 x 8 x 8
                x = self.InceptionV3.Mixed_7c(x)
                # N x 2048 x 8 x 8
                # Adaptive average pooling
                x = F.adaptive_avg_pool2d(x, (1, 1))
                # N x 2048 x 1 x 1
                return x
            
    def calc_fID(self,o,w,bat_size=64):
        # random permute the real & gen data
        
        N = o.size()[0] # No of vectors
        r_ind = torch.randperm(N)
        
        o = o.view(N,-1)[r_ind].view(o.size())
        w = w.view(N,-1)[r_ind].view(w.size())
        
        # send in batches to speed-up computation?
        i=0
        o_l = []
        w_l = []
        while i < N:
            # resize_to_incpV3 returns [bat_size,2048,1,1]
            # we store the tensors in a list
            o_l.append(self.resize_to_incpV3(o[i:i+bat_size].to(device=self.device)).detach().to(device='cpu'))
            w_l.append(self.resize_to_incpV3(w[i:i+bat_size].to(device=self.device)).detach().to(device='cpu'))
            i=i+bat_size
        
        # concatenate the list of tensors into full batch tensor - [N,2048,1,1]
        # And remove 1 dimensions
        # o, w - [N,2048]
        o = torch.squeeze(torch.cat(o_l,0))
        w = torch.squeeze(torch.cat(w_l,0))
        
        # calculate fid
        
        # E[o], E[w]
        m_o = torch.sum(o,dim=0,keepdim=True)/N # dim[1,2048]
        m_w = torch.sum(w,dim=0,keepdim=True)/N # dim[1,2048]
        #print('means')
        #print(m_o.size(),m_w.size())
        
    
        # Covariance Matrix C[o] = E[oo_t] - E[o]E[o_t]
        C_o = torch.matmul(torch.transpose(o,0,1),o)/N - torch.matmul(torch.transpose(m_o,0,1),m_o)
        C_w = torch.matmul(torch.transpose(w,0,1),w)/N - torch.matmul(torch.transpose(m_w,0,1),m_w)
        #print('covar')
        #print(C_o.size(),C_w.size())
    
        a = torch.dist(m_o,m_w).pow(2)
        b = linalg.sqrtm(torch.matmul(C_o,C_w).numpy(),disp=False)[0]
        if np.iscomplexobj(b):
            b = b.real
            b = torch.from_numpy(b).float()
        c = torch.trace(C_o + C_w - 2*b)
        #print(a,c)
        return a+c
        
        

In [ ]:
# instantiate FID object
fid1 = FID('./cache_dir',device)
fid2 = fID(device)
#fid_score1 = fid1.compute_fid(input1,input2)
#print(fid_score1)


In [ ]:
# Tesing fid implementation
input1 = torch.randn(1000,3,32,32)
input2 = torch.randn(1000,3,32,32)
fid2 = fID(device)
fid_score2 = fid2.calc_fID(input1,input2)
print(fid_score2)

In [ ]:
# Testing spectralnormalization impl
w_input = torch.randn(128,256,32,32)
w_out = spectralNorm(w_input)
print(type(w_out))
W_out = spectralNorm(w_out)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # mean = 0.0, std = 0.02
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0)

In [ ]:
# Defining the generator
from torch import nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # input is Z, going into a convolution
        
        self.conv1 = nn.ConvTranspose2d( nz, 512, 4, 1, 0, bias=False) # 4*4
        self.bn1 = nn.BatchNorm2d(512)

        self.conv2 = nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False) # 8*8
        self.bn2 = nn.BatchNorm2d(256)
        self.attn1 = SelfAttention(256,1)

        self.conv3 = nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False) # 16*16
        self.bn3 = nn.BatchNorm2d(128)
        self.attn2 = SelfAttention(128,1)

        self.conv4 = nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False) # 32*32
        
        self.bn4 = nn.BatchNorm2d(3)
        

    def forward(self, input):
        
        self.conv1.weight = spectralNorm(self.conv1.weight)       
        x = self.conv1(input)
        x = self.bn1(x)
        x = nn.ReLU(True)(x)
        
        self.conv2.weight = spectralNorm(self.conv2.weight)
        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.ReLU(True)(x)
        
        x = self.attn1(x) # self attention 1
        
        self.conv3.weight = spectralNorm(self.conv3.weight)
        x = self.conv3(x)
        x = self.bn3(x)
        x = nn.ReLU(True)(x)
        
        x = self.attn2(x) # self attention 2
        
        self.conv4.weight = spectralNorm(self.conv4.weight)
        x = self.conv4(x)
        x = self.bn4(x)
        
        x = nn.Tanh()(x)
        
        return x

In [ ]:
# Create generator & initialize weights
netG = Generator().to(device)
netG.apply(weights_init)
print(netG)

In [ ]:
# testing generator output
# Generate batch of latent vectors
noise1 = torch.randn(batch_size, nz, 1, 1, device=device)
# Generate fake image batch with G
fake1 = netG(noise1)
print(fake1.size())

In [ ]:
# Defining the discriminator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
       
        self.conv1 = nn.Conv2d(nc, 64, 4, 2, 1, bias=False) #16*16
        self.bn0 = nn.BatchNorm2d(64)
       
         
        self.conv2 = nn.Conv2d(64, 128, 4, 2, 1, bias=False) #8*8
        self.bn1 = nn.BatchNorm2d(128) #7*7
       
         
        self.conv3 = nn.Conv2d(128, 256, 4, 1, 1, bias=False) #7*7
        self.bn2 = nn.BatchNorm2d(256)
        
        # self attention layer with k=1 , output = 256/k = 256
        self.attn1 = SelfAttention(256,1)
    
        self.conv4 = nn.Conv2d(256, 512, 4, 1, 1, bias=False) #6*6
        self.bn3 = nn.BatchNorm2d(512)
        
        self.attn2 = SelfAttention(512,1)
      
        self.conv5 = nn.Conv2d(512, 1, 4, 1, 0, bias=False) #3*3
        self.bn4 = nn.BatchNorm2d(1)
       
        self.ln1 = nn.Linear(9,1, bias=False)
       
        

    def forward(self, input):
        
        self.conv1.weight = spectralNorm(self.conv1.weight)
        x = self.conv1(input)
        x = self.bn0(x)
        x = nn.LeakyReLU(0.2, inplace=True)(x)
        
        self.conv2.weight = spectralNorm(self.conv2.weight)
        x = self.conv2(x)
        x = self.bn1(x)
        x = nn.LeakyReLU(0.2, inplace=True)(x)
        
        self.conv3.weight = spectralNorm(self.conv3.weight)
        x = self.conv3(x)
        x = self.bn2(x)
        x = nn.LeakyReLU(0.2, inplace=True)(x)
        
        x = self.attn1(x) # Attention size 256
        
        self.conv4.weight = spectralNorm(self.conv4.weight)
        x = self.conv4(x)
        x = self.bn3(x)
        x = nn.LeakyReLU(0.2, inplace=True)(x)
        
        x = self.attn2(x) # Attention size 512
        
        self.conv5.weight = spectralNorm(self.conv5.weight)
        x = self.conv5(x)
        x = self.bn4(x)
        x = nn.LeakyReLU(0.2, inplace=True)(x)
        
        x = nn.Flatten()(x)   
        x = self.ln1(x)
        x = nn.Sigmoid()(x)
        
        return x

In [ ]:
netD = Discriminator().to(device)
#netD.apply(weights_init)
print(netD)

In [ ]:

# Generate 64 latent z vectors to check generator training
fixed_noise = torch.randn(1000, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=0.0004, betas=(0, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0001, betas=(0, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
fid_scores = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the trainloader
    for i, data in enumerate(trainloader, 0):

        #########################################################################
        # (1) Update D network: minimize E[max(0,(1-D(x)))] + E[max(0,1+D(G(z)))]
        #########################################################################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        
        label = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        
        # dim of output [b_size]
        
        # Calculate loss on all-real batch
        
        # Hinge loss for discriminator 
        # E[max(0,(1-D(x)))] + E[max(0,1+D(G(z)))]
        
        errD_real = torch.mean(torch.nn.ReLU()(1.0 - output)) # E[max(0,(1-D(x)))] 
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = torch.mean(torch.nn.ReLU()(1.0 + output)) # E[max(0,1+D(G(z))]
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: hinge loss for generator = -E(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = -1.0*torch.mean(output) # -E[D(G(z))]
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(trainloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(trainloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu() # 1000 fake images
                real_imgs = next(iter(testloader)) # 1000 real images
                #fid_score = fid1.compute_fid(real_imgs[0],fake)
                fid_score = fid2.calc_fID(real_imgs[0],fake) # This step takes 2 mins
                print('fid_score: %.4f'%(fid_score))
                fid_scores.append(fid_score) # save fid scores
                
            img_list.append(vutils.make_grid(fake[-64:], padding=2, normalize=True))

        iters += 1